In [ ]:

#folder1 = "C:/Users/HP14/Desktop/Electronics/damn/images"
#folder2 = "C:/Users/HP14/Desktop/Electronics/damn/labels"

import os

def find_complement_intersection_names(folder1, folder2):
  """
  This function finds the complement of the intersection of filenames (excluding extensions) in two folders.

  Args:
      folder1 (str): Path to the first folder.
      folder2 (str): Path to the second folder.

  Returns:
      set: Set of filenames (without extensions) that are in one folder but not the other (complement of intersection).
  """
  all_filenames = set(os.path.splitext(f)[0] for f in os.listdir(folder1)) | set(os.path.splitext(f)[0] for f in os.listdir(folder2))
  intersection = all_filenames.intersection(set(os.path.splitext(f)[0] for f in os.listdir(folder1))) & all_filenames.intersection(set(os.path.splitext(f)[0] for f in os.listdir(folder2)))
  complement = all_filenames - intersection

  return complement

# Example usage
folder1 = "C:/Users/HP14/Desktop/Electronics/damn/images"
folder2 = "C:/Users/HP14/Desktop/Electronics/damn/labels"

complement_names = find_complement_intersection_names(folder1, folder2)

if complement_names:
  print("Complement of intersection (filenames without extensions in one folder but not the other):")
  for filename in complement_names:
    print(filename)
else:
  print("No filenames found in one folder but not the other")

print("Finished checking files.")


 

No filenames found in one folder but not the other
Finished checking files.


In [ ]:
import os
os.chdir("C:/Users/HP14/Desktop/Electronics")


In [2]:
from ultralytics import YOLO
import cv2
import math
import time
import os
from datetime import datetime

# start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# model
model = YOLO("BestModel.pt")

# object classes
classNames = ["gun", "knife"]

# confidence threshold
confidence_threshold = 0.5

# specify the directory to save screenshots
screenshot_directory = "screenshots/"

# create the directory if it doesn't exist
if not os.path.exists(screenshot_directory):
    os.makedirs(screenshot_directory)

while True:
    success, img = cap.read()
    if not success:
        break

    results = model(img, stream=True)

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

            # confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100
            print("Confidence --->", confidence)

            # class name
            cls = int(box.cls[0])
            print("Class name -->", classNames[cls])

            # if confidence is above the threshold, save a screenshot
            if confidence > confidence_threshold:
                current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                screenshot_filename = f"screenshot_{classNames[cls]}_{current_time}.jpg"
                full_path = os.path.join(screenshot_directory, screenshot_filename)
                cv2.imwrite(full_path, img)
                print(f"Screenshot saved as {full_path}")

            # prepare text for display (class name + confidence)
            label = f"{classNames[cls]}: {confidence:.2f}"

            # object details for text placement
            org = [x1, y1 - 10]  # position text above the bounding box
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 0.9
            color = (255, 255, 255)  # white text
            thickness = 2

            # put the label (class name + confidence) on the image
            cv2.putText(img, label, org, font, fontScale, color, thickness)

    # display webcam feed
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 344.5ms
Speed: 7.8ms preprocess, 344.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 358.5ms
Speed: 6.1ms preprocess, 358.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 383.5ms
Speed: 3.4ms preprocess, 383.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 322.3ms
Speed: 5.3ms preprocess, 322.3ms inference, 9.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 301.5ms
Speed: 6.1ms preprocess, 301.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 283.1ms
Speed: 3.5ms preprocess, 283.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 267.4ms
Speed: 3.5ms preprocess, 267.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 282.8ms
Speed: 4.0ms prepr

KeyboardInterrupt: 

In [1]:
from ultralytics import YOLO
import cv2
import math
import os
from datetime import datetime
import requests
import json

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Load YOLO model
model = YOLO("BestModel.pt")

# Object classes
classNames = ["gun", "knife"]

# Confidence threshold
confidence_threshold = 0.5

# Specify the directory to save screenshots
screenshot_directory = "screenshots/"

# Create the directory if it doesn't exist
if not os.path.exists(screenshot_directory):
    os.makedirs(screenshot_directory)

# API endpoint
api_url = "http://localhost:7125/api/DetectAPI"

def get_binary_object_type(class_name):
    if class_name == "gun":
        return 0 
    else:
        return 1

while True:
    success, img = cap.read()
    if not success:
        break

    results = model(img, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw rectangle
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

            # Confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100
            print("Confidence --->", confidence)

            # Class name and binary representation
            cls = int(box.cls[0])
            class_name = classNames[cls]
            binary_type = get_binary_object_type(class_name)
            print(f"Class name --> {class_name}")

            if confidence > confidence_threshold:
                current_time = datetime.now().isoformat()
                screenshot_filename = f"screenshot_{class_name}_{current_time.replace(':', '-')}.jpg"
                full_path = os.path.join(screenshot_directory, screenshot_filename)
                cv2.imwrite(full_path, img)
                print(f"Screenshot saved as {full_path}")

                # Prepare data to send to API
                data = {
                    "WeaponType": binary_type,  # 0 for gun, 1 for knife
                    "Confidence": confidence,
                    "Timestamp": current_time
                }

                # Send data to API
                try:
                    headers = {'Content-Type': 'application/json'}
                    response = requests.post(api_url, json=data, headers=headers)
                    if response.status_code == 201:  # 201 is the status code for "Created"
                        print("Data sent to API successfully")
                    else:
                        print(f"Failed to send data to API. Status code: {response.status_code}")
                        print(f"Response content: {response.text}")
                except requests.exceptions.RequestException as e:
                    print(f"Error sending data to API: {e}")

            # Prepare label
            label = f"{class_name} ({binary_type}): {confidence:.2f}"

            # Put label on image
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Display webcam feed
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 308.5ms
Speed: 15.4ms preprocess, 308.5ms inference, 11.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 262.9ms
Speed: 6.5ms preprocess, 262.9ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 211.0ms
Speed: 2.5ms preprocess, 211.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 211.2ms
Speed: 4.1ms preprocess, 211.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 228.4ms
Speed: 2.3ms preprocess, 228.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 202.3ms
Speed: 3.1ms preprocess, 202.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 207.7ms
Speed: 2.7ms preprocess, 207.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 193.2ms
Speed: 3.1ms pre

KeyboardInterrupt: 

: 

In [1]:
from ultralytics import YOLO
import cv2
import math
import os
from datetime import datetime
import requests
import json
import logging
import urllib3

# Disable InsecureRequestWarning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Load YOLO model
model = YOLO("BestModel.pt")

# Object classes
classNames = ["gun", "knife"]

# Confidence threshold
confidence_threshold = 0.5

# Specify the directory to save screenshots
screenshot_directory = "screenshots/"

# Create the directory if it doesn't exist
if not os.path.exists(screenshot_directory):
    os.makedirs(screenshot_directory)

# API endpoint
api_url = "https://localhost:7125/api/DetectAPI"

def get_binary_object_type(class_name):
    if class_name == "gun": 
        return False
    else:
        return True

while True:
    success, img = cap.read()
    if not success:
        logging.error("Failed to capture image from webcam")
        break

    results = model(img, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw rectangle
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

            # Confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100
            logging.info(f"Confidence: {confidence}")

            # Class name and binary representation
            cls = int(box.cls[0])
            class_name = classNames[cls]
            binary_type = get_binary_object_type(class_name)
            logging.info(f"Class name: {class_name}")

            if confidence > confidence_threshold:
                current_time = datetime.now().isoformat()
                screenshot_filename = f"screenshot_{class_name}_{current_time.replace(':', '-')}.jpg"
                full_path = os.path.join(screenshot_directory, screenshot_filename)
                cv2.imwrite(full_path, img)
                logging.info(f"Screenshot saved as {full_path}")

                # Prepare data to send to API
                data = {
                "WeaponType": get_binary_object_type(class_name),
                "Confidence": float(confidence),
                "Timestamp": current_time
}

                # Send data to API
                try:
                    headers = {'Content-Type': 'application/json'}
                    logging.info(f"Sending data to API: {json.dumps(data, indent=2)}")
                    response = requests.post(api_url, json=data, headers=headers, verify=False)  # Added verify=False
                    logging.info(f"Status Code: {response.status_code}")
                    logging.info(f"Response Body: {response.text}")
                    logging.info(f"Response Headers: {response.headers}")
                    
                    if response.status_code == 201:  # 201 is the status code for "Created"
                        logging.info("Data sent to API successfully")
                    else:
                        logging.error(f"Failed to send data to API. Status code: {response.status_code}")
                        logging.error(f"Response content: {response.text}")
                except requests.exceptions.RequestException as e:
                    logging.error(f"Error sending data to API: {e}")
                except Exception as e:
                    logging.error(f"Unexpected error: {e}")

            # Prepare label
            label = f"{class_name} ({binary_type}): {confidence:.2f}"

            # Put label on image
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Display webcam feed
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 776.4ms
Speed: 23.7ms preprocess, 776.4ms inference, 18.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 665.4ms
Speed: 32.4ms preprocess, 665.4ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 559.0ms
Speed: 9.1ms preprocess, 559.0ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 461.7ms
Speed: 4.3ms preprocess, 461.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 452.8ms
Speed: 5.2ms preprocess, 452.8ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 497.0ms
Speed: 8.0ms preprocess, 497.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 482.3ms
Speed: 4.4ms preprocess, 482.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 461.4ms
Speed: 5.3ms pr

2024-09-22 11:34:08,622 - INFO - Confidence: 0.38
2024-09-22 11:34:08,622 - INFO - Class name: knife


Speed: 7.8ms preprocess, 372.2ms inference, 25.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 knife, 452.0ms


2024-09-22 11:34:09,098 - INFO - Confidence: 0.66
2024-09-22 11:34:09,114 - INFO - Class name: knife
2024-09-22 11:34:09,146 - INFO - Screenshot saved as screenshots/screenshot_knife_2024-09-22T11-34-09.114031.jpg
2024-09-22 11:34:09,147 - INFO - Sending data to API: {
  "WeaponType": true,
  "Confidence": 0.66,
  "Timestamp": "2024-09-22T11:34:09.114031"
}
2024-09-22 11:34:14,292 - INFO - Status Code: 201
2024-09-22 11:34:14,292 - INFO - Response Body: {"id":52,"cameraID":1,"camera":null,"timestamp":"2024-09-22T11:34:09.114031","weaponType":true,"confidence":0.66,"status":0,"reason":"Gun is detected","userID":"4f394d62-c6c1-4604-b44d-a640fb3d6b07","user":null}
2024-09-22 11:34:14,296 - INFO - Response Headers: {'Content-Type': 'application/json; charset=utf-8', 'Date': 'Sun, 22 Sep 2024 08:34:14 GMT', 'Server': 'Kestrel', 'Location': 'https://localhost:7125/api/DetectAPI/52', 'Transfer-Encoding': 'chunked'}
2024-09-22 11:34:14,299 - INFO - Data sent to API successfully


Speed: 8.0ms preprocess, 452.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 453.3ms
Speed: 14.1ms preprocess, 453.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 knife, 529.0ms


...

2024-09-22 11:34:17,584 - INFO - Confidence: 0.56
2024-09-22 11:34:17,584 - INFO - Class name: knife
2024-09-22 11:34:17,601 - INFO - Screenshot saved as screenshots/screenshot_knife_2024-09-22T11-34-17.593177.jpg
2024-09-22 11:34:17,601 - INFO - Sending data to API: {
  "WeaponType": true,
  "Confidence": 0.56,
  "Timestamp": "2024-09-22T11:34:17.593177"
}
2024-09-22 11:34:17,867 - INFO - Status Code: 201
2024-09-22 11:34:17,869 - INFO - Response Body: {"id":53,"cameraID":1,"camera":null,"timestamp":"2024-09-22T11:34:17.593177","weaponType":true,"confidence":0.56,"status":0,"reason":"Gun is detected","userID":"4f394d62-c6c1-4604-b44d-a640fb3d6b07","user":null}
2024-09-22 11:34:17,869 - INFO - Response Headers: {'Content-Type': 'application/json; charset=utf-8', 'Date': 'Sun, 22 Sep 2024 08:34:17 GMT', 'Server': 'Kestrel', 'Location': 'https://localhost:7125/api/DetectAPI/53', 'Transfer-Encoding': 'chunked'}
2024-09-22 11:34:17,869 - INFO - Data sent to API successfully


Speed: 3.0ms preprocess, 392.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 gun, 528.7ms


2024-09-22 11:34:18,437 - INFO - Confidence: 0.56
2024-09-22 11:34:18,437 - INFO - Class name: gun
2024-09-22 11:34:18,463 - INFO - Screenshot saved as screenshots/screenshot_gun_2024-09-22T11-34-18.437845.jpg
2024-09-22 11:34:18,466 - INFO - Sending data to API: {
  "WeaponType": false,
  "Confidence": 0.56,
  "Timestamp": "2024-09-22T11:34:18.437845"
}
2024-09-22 11:34:18,667 - INFO - Status Code: 201
2024-09-22 11:34:18,673 - INFO - Response Body: {"id":54,"cameraID":1,"camera":null,"timestamp":"2024-09-22T11:34:18.437845","weaponType":false,"confidence":0.56,"status":0,"reason":"Knife is detected","userID":"4f394d62-c6c1-4604-b44d-a640fb3d6b07","user":null}
2024-09-22 11:34:18,673 - INFO - Response Headers: {'Content-Type': 'application/json; charset=utf-8', 'Date': 'Sun, 22 Sep 2024 08:34:18 GMT', 'Server': 'Kestrel', 'Location': 'https://localhost:7125/api/DetectAPI/54', 'Transfer-Encoding': 'chunked'}
2024-09-22 11:34:18,673 - INFO - Data sent to API successfully


Speed: 11.2ms preprocess, 528.7ms inference, 6.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 462.6ms
Speed: 8.0ms preprocess, 462.6ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 418.8ms
Speed: 0.0ms preprocess, 418.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 462.5ms
Speed: 3.5ms preprocess, 462.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 419.2ms
Speed: 2.3ms preprocess, 419.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 477.5ms
Speed: 9.1ms preprocess, 477.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 398.4ms
Speed: 7.5ms preprocess, 398.4ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 448.3ms
Speed: 2.1ms preprocess, 448.3ms inference, 0.0ms post

KeyboardInterrupt: 

In [2]:
from ultralytics import YOLO
import cv2
import math
import os
from datetime import datetime, timedelta
import requests
import json
import logging
import urllib3

# Disable InsecureRequestWarning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Load YOLO model
model = YOLO("BestModel.pt")

# Object classes
classNames = ["gun", "knife"]

# Confidence threshold
confidence_threshold = 0.5

# Specify the directory to save screenshots
screenshot_directory = "screenshots/"

# Create the directory if it doesn't exist
if not os.path.exists(screenshot_directory):
    os.makedirs(screenshot_directory)

# API endpoint
api_url = "https://localhost:7125/api/DetectAPI"

def get_binary_object_type(class_name):
    if class_name == "gun": 
        return False
    else:
        return True

# Initialize variables for best detection
best_detection = None
last_sent_time = datetime.now()
detection_interval = timedelta(seconds=10)

while True:
    success, img = cap.read()
    if not success:
        logging.error("Failed to capture image from webcam")
        break

    results = model(img, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw rectangle
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

            # Confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100
            logging.info(f"Confidence: {confidence}")

            # Class name and binary representation
            cls = int(box.cls[0])
            class_name = classNames[cls]
            binary_type = get_binary_object_type(class_name)
            logging.info(f"Class name: {class_name}")

            if confidence > confidence_threshold:
                # Update best detection if this one has higher confidence
                if best_detection is None or confidence > best_detection['Confidence']:
                    best_detection = {
                        "WeaponType": binary_type,
                        "Confidence": float(confidence),
                        "Timestamp": datetime.now().isoformat(),
                        "Image": img.copy(),
                        "ClassName": class_name
                    }

            # Prepare label
            label = f"{class_name} ({binary_type}): {confidence:.2f}"

            # Put label on image
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Check if it's time to send the best detection
    current_time = datetime.now()
    if current_time - last_sent_time >= detection_interval and best_detection is not None:
        # Save screenshot
        screenshot_filename = f"screenshot_{best_detection['ClassName']}_{best_detection['Timestamp'].replace(':', '-')}.jpg"
        full_path = os.path.join(screenshot_directory, screenshot_filename)
        cv2.imwrite(full_path, best_detection['Image'])
        logging.info(f"Screenshot saved as {full_path}")

        # Prepare data to send to API
        data = {
            "WeaponType": best_detection['WeaponType'],
            "Confidence": best_detection['Confidence'],
            "Timestamp": best_detection['Timestamp']
        }

        # Send data to API
        try:
            headers = {'Content-Type': 'application/json'}
            logging.info(f"Sending data to API: {json.dumps(data, indent=2)}")
            response = requests.post(api_url, json=data, headers=headers, verify=False)  # Added verify=False
            logging.info(f"Status Code: {response.status_code}")
            logging.info(f"Response Body: {response.text}")
            logging.info(f"Response Headers: {response.headers}")
            
            if response.status_code == 201:  # 201 is the status code for "Created"
                logging.info("Data sent to API successfully")
            else:
                logging.error(f"Failed to send data to API. Status code: {response.status_code}")
                logging.error(f"Response content: {response.text}")
        except requests.exceptions.RequestException as e:
            logging.error(f"Error sending data to API: {e}")
        except Exception as e:
            logging.error(f"Unexpected error: {e}")

        # Reset for next interval
        best_detection = None
        last_sent_time = current_time

    # Display webcam feed
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 653.4ms
Speed: 5.1ms preprocess, 653.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 562.5ms
Speed: 6.4ms preprocess, 562.5ms inference, 6.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 551.7ms
Speed: 8.3ms preprocess, 551.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 493.8ms
Speed: 5.4ms preprocess, 493.8ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 480.6ms
Speed: 6.0ms preprocess, 480.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 480.4ms
Speed: 8.1ms preprocess, 480.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 505.1ms
Speed: 3.4ms preprocess, 505.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 509.2ms
Speed: 4.9ms prepr

2024-09-22 11:50:03,066 - INFO - Confidence: 0.27
2024-09-22 11:50:03,068 - INFO - Class name: gun


Speed: 3.9ms preprocess, 405.1ms inference, 10.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 693.8ms
Speed: 13.8ms preprocess, 693.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 542.6ms
Speed: 8.0ms preprocess, 542.6ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 524.1ms
Speed: 6.2ms preprocess, 524.1ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 485.8ms
Speed: 7.9ms preprocess, 485.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 knife, 410.2ms


2024-09-22 11:50:05,877 - INFO - Confidence: 0.75
2024-09-22 11:50:05,879 - INFO - Class name: knife


Speed: 5.6ms preprocess, 410.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)


...

2024-09-22 11:51:46,840 - INFO - Confidence: 0.55
2024-09-22 11:51:46,842 - INFO - Class name: knife


Speed: 1.2ms preprocess, 462.4ms inference, 10.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 407.6ms
Speed: 6.1ms preprocess, 407.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 430.6ms
Speed: 6.6ms preprocess, 430.6ms inference, 10.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 446.0ms
Speed: 2.5ms preprocess, 446.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 530.4ms
Speed: 7.4ms preprocess, 530.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 406.6ms
Speed: 4.8ms preprocess, 406.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 494.1ms
Speed: 2.4ms preprocess, 494.1ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 610.0ms
Speed: 7.4ms preprocess, 610.0ms inference, 0.0ms pos

2024-09-22 11:51:54,874 - INFO - Screenshot saved as screenshots/screenshot_knife_2024-09-22T11-51-46.350949.jpg
2024-09-22 11:51:54,876 - INFO - Sending data to API: {
  "WeaponType": true,
  "Confidence": 0.67,
  "Timestamp": "2024-09-22T11:51:46.350949"
}
2024-09-22 11:51:55,039 - INFO - Status Code: 201
2024-09-22 11:51:55,039 - INFO - Response Body: {"id":61,"cameraID":1,"camera":null,"timestamp":"2024-09-22T11:51:46.350949","weaponType":true,"confidence":0.67,"status":0,"reason":"Gun is detected","userID":"4f394d62-c6c1-4604-b44d-a640fb3d6b07","user":null}
2024-09-22 11:51:55,039 - INFO - Response Headers: {'Content-Type': 'application/json; charset=utf-8', 'Date': 'Sun, 22 Sep 2024 08:51:54 GMT', 'Server': 'Kestrel', 'Location': 'https://localhost:7125/api/DetectAPI/61', 'Transfer-Encoding': 'chunked'}
2024-09-22 11:51:55,045 - INFO - Data sent to API successfully



0: 480x640 (no detections), 705.7ms
Speed: 9.8ms preprocess, 705.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 660.7ms
Speed: 6.9ms preprocess, 660.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 770.7ms
Speed: 6.7ms preprocess, 770.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 670.4ms
Speed: 8.1ms preprocess, 670.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 681.1ms
Speed: 6.0ms preprocess, 681.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 752.5ms
Speed: 11.0ms preprocess, 752.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 602.8ms
Speed: 6.5ms preprocess, 602.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 632.2ms
Speed: 7.4ms prep

KeyboardInterrupt: 